# Lab 4, Exercise 1

In [15]:
import numpy as np
import pandas as pd

## Load data 

Here is an example of the first couple rows from the data:

| id | dur       | proto | service | state | spkts | dpkts | sbytes | dbytes | rate        | sttl | dttl | sload     | dload | sloss | dloss | sinpkt | dinpkt | sjit | djit | swin | stcpb | dtcpb | dwin | tcprtt | synack | ackdat | smean | dmean | trans\_depth | response\_body\_len | ct\_srv\_src | ct\_state\_ttl | ct\_dst\_ltm | ct\_src\_dport\_ltm | ct\_dst\_sport\_ltm | ct\_dst\_src\_ltm | is\_ftp\_login | ct\_ftp\_cmd | ct\_flw\_http\_mthd | ct\_src\_ltm | ct\_srv\_dst | is\_sm\_ips\_ports | attack\_cat | label |
|----|-----------|-------|---------|-------|-------|-------|--------|--------|-------------|------|------|-----------|-------|-------|-------|--------|--------|------|------|------|-------|-------|------|--------|--------|--------|-------|-------|--------------|---------------------|--------------|----------------|--------------|---------------------|---------------------|-------------------|----------------|--------------|---------------------|--------------|--------------|--------------------|-------------|-------|
| 1  | 0\.000011 | udp   | \-      | INT   | 2     | 0     | 496    | 0      | 90909\.0902 | 254  | 0    | 180363632 | 0     | 0     | 0     | 0\.011 | 0      | 0    | 0    | 0    | 0     | 0     | 0    | 0      | 0      | 0      | 248   | 0     | 0            | 0                   | 2            | 2              | 1            | 1                   | 1                   | 2                 | 0              | 0            | 0                   | 1            | 2            | 0                  | Normal      | 0     |


In [16]:
# Load the training and test data corresponding to exercise 1
# Create two separate pandas dataframes for the training and test data
# For each dataframe, import the following CSV data
# training set: 'data/exercise1/UNSW_NB15_training-set.csv'
# test set: 'data/exercise1/UNSW_NB15_testing-set.csv'

df_train = pd.read_csv('data/exercise1/UNSW_NB15_training-set.csv')
df_test = pd.read_csv('data/exercise1/UNSW_NB15_testing-set.csv')

### Part 1 
Keep the two dataframes separate and create train/test data and labels.  This will be used to experiment with the case where there are different types of activities in the training versus test set.

In [17]:
# Do all the following operations in the cell on both the dataframes separately
# 1) Keep only the datapoints where the 'attack_cat' column is equal to either 'Normal' or 'Fuzzers'
# CODE HERE
df_train = df_train[df_train['attack_cat'].isin(['Normal', 'Fuzzers'])]
df_test = df_test[df_test['attack_cat'].isin(['Normal', 'Fuzzers'])]

# 2) Get the labels from the dataframe (i.e., the values in the 'attack_cat' column)
# CODE HERE
labels_train = df_train['attack_cat'].values
labels_test = df_test['attack_cat'].values

# 3) Keep only the features we care about for this experiment.
# We only care about the numerical features between column 'spkts' and 'is_sm_ips_ports' (inclusive)
# CODE HERE
features_train = df_train.iloc[:, df_train.columns.get_loc('spkts'):df_train.columns.get_loc('is_sm_ips_ports')+1].values
features_test = df_test.iloc[:, df_test.columns.get_loc('spkts'):df_test.columns.get_loc('is_sm_ips_ports')+1].values

# You should now have four inputs usable for scikit-learn:
# training data
# training labels
# test data
# test labels
# Hint: You may have to add some minor code in the above to get the data ready for scikit-learn

In [18]:
from sklearn.ensemble import RandomForestClassifier
rfc_train = RandomForestClassifier()
rfc_train.fit(features_train, labels_train)
predict = rfc_train.predict(features_train)

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
print('Metrics on training data:')
print('Precision: ', precision_score(labels_train, predict, average='weighted'))
print('Recall: ', recall_score(labels_train, predict, average='weighted'))
print('F1: ', f1_score(labels_train, predict, average='weighted'))
print('Accuracy: ', accuracy_score(labels_train, predict))

predict = rfc_train.predict(features_test)

print('\nMetrics on test data:')
print('Precision: ', precision_score(labels_test, predict, average='weighted'))
print('Recall: ', recall_score(labels_test, predict, average='weighted'))
print('F1: ', f1_score(labels_test, predict, average='weighted'))
print('Accuracy: ', accuracy_score(labels_test, predict))

Metrics on training data:
Precision:  0.9998606632343643
Recall:  0.9998606660164414
F1:  0.9998606467896146
Accuracy:  0.9998606660164414

Metrics on test data:
Precision:  0.8158847314454788
Recall:  0.7986493044322226
F1:  0.7470410366701024
Accuracy:  0.7986493044322226


### Part 2 
Create a new training/test split by combining the dataframes into one.  Then split the dataframe randomly into train/test data and labels.  This will be used to experiment with the case where there are largely the same types of activities in the training and test set.

In [19]:
# Combine the dataframes into a single dataframe, then do the following
# CODE HERE
df = pd.concat([df_train, df_test])

# 1) Keep only the datapoints where the 'attack_cat' column is equal to either 'Normal' or 'Fuzzers'
# CODE HERE
df = df[df['attack_cat'].isin(['Normal', 'Fuzzers'])]

# 2) Get the labels from the dataframe (i.e., the values in the 'attack_cat' column)
# CODE HERE
labels = df['attack_cat'].values

# 3) Keep only features we care about for this experiment.
# We only care about the numerical features between column 'spkts' and 'is_sm_ips_ports' (inclusive)
# CODE HERE
features = df.iloc[:, df.columns.get_loc('spkts'):df.columns.get_loc('is_sm_ips_ports')+1].values

# 4) Create a random split; put 50% of the data into the training set and the other 50% into the test set
# Use scikit-learn's 'train_test_split'
# Hint: You may have to add some minor code in the above to get the data ready for scikit-learn's 'train_test_split'
# CODE HERE
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.5)

# You should now have four inputs usable for scikit-learn that are distinct from the inputs you created in Part 1.
# The inputs should correspond to:
# training data
# training labels
# test data
# test labels

## Train a classifier

In [20]:
# For each of the train/test splits, create a separate random forest with default sklearn parameters
# Hint: Ignoring import statements, each random forest can be created in a single line of code
# CODE HERE
from sklearn.ensemble import RandomForestClassifier
rfc_train = RandomForestClassifier()
rfc_train.fit(features_train, labels_train)

RandomForestClassifier()

## Inference and results

In [21]:
# For each of the train/test splits and associated random forest, do the following:

# 1) Predict labels on the training data
# CODE HERE
predict = rfc_train.predict(features_train)

# 2) Print metrics on the training data; use sklearn's implementation of precision, recall, f1, and accuracy
# CODE HERE
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
print('Metrics on training data:')
print('Precision: ', precision_score(labels_train, predict, average='weighted'))
print('Recall: ', recall_score(labels_train, predict, average='weighted'))
print('F1: ', f1_score(labels_train, predict, average='weighted'))
print('Accuracy: ', accuracy_score(labels_train, predict))

# 3) Predict labels on the test data
# CODE HERE
predict = rfc_train.predict(features_test)

# 4) Print metrics on the test data; again, use precision, recall, f1, and accuracy
# CODE HERE
print('\nMetrics on test data:')
print('Precision: ', precision_score(labels_test, predict, average='weighted'))
print('Recall: ', recall_score(labels_test, predict, average='weighted'))
print('F1: ', f1_score(labels_test, predict, average='weighted'))
print('Accuracy: ', accuracy_score(labels_test, predict))

Metrics on training data:
Precision:  0.9956310058043459
Recall:  0.9956331132831824
F1:  0.9956319171995252
Accuracy:  0.9956331132831824

Metrics on test data:
Precision:  0.8995047506717183
Recall:  0.9016938744178906
F1:  0.90032090702092
Accuracy:  0.9016938744178906


## Questions:

1) For results using Part 1 data, what is the precision and recall?

Precision: 0.82
Recall: 0.80

2) For results using Part 1 data, describe the difference in the results on the training and test data. What does this signify? 

The training data has a higher precision and recall than the test data. This signifies that the model is overfits the training data, while not generalizing well to the test data.

3) What changes in the results on the test data once you combine the data for Part 2? Does this produce a better classifier? Why or why not?

Precision: 0.90
Recall: 0.90

The results on the test data are better once the data is combined for Part 2. This produces a better classifier because the model is trained on a more diverse set of data, and is able to generalize better to the test data.